<a href="https://www.kaggle.com/code/aleksandrmorozov123/cv-object-detection?scriptVersionId=198122471" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/brain-tumor-object-detection-datasets/coronal_t1wce_2_class/coronal_t1wce_2_class.yaml
/kaggle/input/brain-tumor-object-detection-datasets/coronal_t1wce_2_class/labels/test/55 (2).txt
/kaggle/input/brain-tumor-object-detection-datasets/coronal_t1wce_2_class/labels/test/40.txt
/kaggle/input/brain-tumor-object-detection-datasets/coronal_t1wce_2_class/labels/test/44.txt
/kaggle/input/brain-tumor-object-detection-datasets/coronal_t1wce_2_class/labels/test/36.txt
/kaggle/input/brain-tumor-object-detection-datasets/coronal_t1wce_2_class/labels/test/47.txt
/kaggle/input/brain-tumor-object-detection-datasets/coronal_t1wce_2_class/labels/test/19.txt
/kaggle/input/brain-tumor-object-detection-datasets/coronal_t1wce_2_class/labels/test/45.txt
/kaggle/input/brain-tumor-object-detection-datasets/coronal_t1wce_2_class/labels/test/57 (2).txt
/kaggle/input/brain-tumor-object-detection-datasets/coronal_t1wce_2_class/labels/test/59.txt
/kaggle/input/brain-tumor-object-detection-datasets/co

**Brain tumor object detection**

In [2]:
import os
data_folder = "/kaggle/input/brain-tumor-object-detection-datasets"
data_folder_yolo = data_folder + "_yolo"
data_folder_coco = data_folder + "_coco"
folders = os.listdir ("/kaggle/input/brain-tumor-object-detection-datasets")
print (folders)

['coronal_t1wce_2_class', 'axial_t1wce_2_class', 'sagittal_t1wce_2_class']


In [3]:
!sudo apt-get install tree
!tree "/kaggle/input/brain-tumor-object-detection-datasets"/ -d




tree is already the newest version (1.8.0-1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
/kaggle/input/brain-tumor-object-detection-datasets/
|-- axial_t1wce_2_class
|   |-- images
|   |   |-- test
|   |   `-- train
|   `-- labels
|       |-- test
|       `-- train
|-- coronal_t1wce_2_class
|   |-- images
|   |   |-- test
|   |   `-- train
|   `-- labels
|       |-- test
|       `-- train
`-- sagittal_t1wce_2_class
    |-- images
    |   |-- test
    |   `-- train
    `-- labels
        |-- test
        `-- train

21 directories


In [4]:
from glob import glob
def count_yolo_data(folder):
    for images_labels in ["images", "labels"]:
        for train_test in ["train", "test"]:
            file_ext = "*.jpg" if images_labels == "images" else "*.txt"
            p = os.path.join(folder, images_labels, train_test, file_ext)
            files = glob(p)
            print(train_test, images_labels, len(files))

In [5]:
for folder in folders:
    print("-"*8 + folder + "-"*8)
    folder = os.path.join(data_folder, folder)
    count_yolo_data(folder)

--------coronal_t1wce_2_class--------
train images 319
test images 78
train labels 318
test labels 78
--------axial_t1wce_2_class--------
train images 310
test images 75
train labels 296
test labels 75
--------sagittal_t1wce_2_class--------
train images 264
test images 70
train labels 264
test labels 70
